# Controlled

The controlled protocol lets you request a 'controlled' version of a bloq.

A key feature of any program is the ability to do branching. That is, choose what operations to do based on input. The primitive branching feature in quantum algorithms is the idea of a controlled bloq.

In its simplest form, a control bit is specified as an (additional) input to a bloq and the bloq is active when the control input is in the $|1\rangle$ state. Otherwise, the bloq's operation is not performed (said another way: the Identity bloq is performed). The control input can be in superposition. 

In [ ]:
from qualtran.drawing import show_bloq

## `ControlledBloq`

`ControlledBloq(subbloq)` represents a controlled version of `subbloq`. Its decompose method will call `subbloq`'s decompose and wrap each of the child bloqs in `ControlledBloq`.

In [ ]:
from qualtran import ControlledBloq
from qualtran.bloqs.for_testing import TestAtom

bloq = ControlledBloq(subbloq=TestAtom())
show_bloq(bloq)

### Controlled Serial Bloq

In [ ]:
from qualtran.bloqs.for_testing import TestSerialCombo

bloq = ControlledBloq(subbloq=TestSerialCombo())
show_bloq(bloq)
show_bloq(bloq.decompose_bloq())

### Controlled Parallel Bloq

In [ ]:
from qualtran.bloqs.for_testing import TestParallelCombo

bloq = ControlledBloq(subbloq=TestParallelCombo())
show_bloq(bloq)
show_bloq(bloq.decompose_bloq())